In [449]:
#default_exp buffer

In [475]:
#export
from fastcore.basics import basic_repr, store_attr, patch
from fastcore.test import test_eq, test_is

## Buffer

The most important data structure in a text editor is the `Buffer`. A `Buffer` stores text (internally as a list of `lines` which are strings) and allows efficienct insertion and deletion operations.

In [485]:
#export
class Buffer:
    def __init__(self, lines, name=None):
        store_attr()

    def __repr__(self):
        return "\n".join(self.lines)

    def __eq__(self, o):
        if isinstance(o, self.__class__):
            return self.lines == o.lines
        return False

    @classmethod
    def from_string(cls, s):
        return cls(s.splitlines())

Create a `Buffer` by either manually passing a list of lines to the constructor, or from a single string with `from_string`. The original contents of the string can be recovered via the `repr`.

In [486]:
b1 = Buffer(["foo", "bar"])
test_eq(b1.lines, ["foo", "bar"])
test_eq(repr(b1), "foo\nbar")

b2 = Buffer.from_string("foo\nbar")
test_eq(b1, b2)

Insert a string at a given position (`row` and `col`) using `insert`. Since `Buffer`s are _immutable_, `insert` returns a new `Buffer` with the original remaining unmodified.

In [487]:
#export
@patch
def insert(self:Buffer, row, column, s):
    lines = self.lines.copy()
    cur = lines.pop(row)
    before, after = cur[:column], cur[column:]
    if s == "\n":
        lines.insert(row, before)
        lines.insert(row+1, after)
    else:
        lines.insert(row, before+s+after)
    return Buffer(lines, self.name)

In [488]:
b = Buffer(["foo", "bar"])
b2 = b.insert(0, 2, "baz")

test_eq(b.lines, ["foo", "bar"])
test_eq(b2.lines, ["fobazo", "bar"])

Inserting a newline character, `"\n"`, splits the line:

In [489]:
b = Buffer(["foo", "bar"])
test_eq(b.insert(0, 2, "\n").lines, ["fo", "o", "bar"])

---

**TODO:** This doesn't feel like it belongs here. Either belongs in `core` or maybe in `Region`?

Although `Buffer`s operate on `(row, col)` positions, it is often required to move forward or backward `n` steps from a given position. This is handled with the `rel2abs` function, which converts _relative_ movement (number of steps `n`) at a given position in a buffer to a new absolute position.

In other words, it converts from `(row_0, col_0, n)` to `(row_1, col_1)`.

Positive `n` moves forward `n` characters, and negative moves backward. `rel2abs` also handles movement across lines in either direction.

In [490]:
def rel2abs(lines, row, col, n):
    col += n
    if 0 <= col <= len(lines[row]):
        return row, col
    if n < 0:
        return rel2abs(lines, row-1, len(lines[row-1]), col+1)
    if n > 0:
        return rel2abs(lines, row+1, 0, col-len(lines[row])-1)
    raise AssertionError()

In [491]:
lines = ["two", "three", "four"]

test_eq(rel2abs(lines, 2, 2, -2), (2, 0))
test_eq(rel2abs(lines, 2, 2, -3), (1, 5))
test_eq(rel2abs(lines, 2, 2, -5), (1, 3))
test_eq(rel2abs(lines, 2, 2, -10), (0, 2))

test_eq(rel2abs(lines, 2, 0, 2), (2, 2))
test_eq(rel2abs(lines, 1, 5, 3), (2, 2))
test_eq(rel2abs(lines, 1, 3, 5), (2, 2))
test_eq(rel2abs(lines, 0, 2, 10), (2, 2))

---

Delete `n` characters from position `(row, col)` using `delete`, forward if `n` is positive and backward if `n` is negative. Deleting across lines joins lines as necessary.

In [492]:
#export
@patch
def delete(self:Buffer, row, col, n):
    to = rel2abs(self.lines, row, col, n)
    (r1, c1), (r2, c2) = sorted(((row, col), to))
    before = self.lines[:r1]
    cur = self.lines[r1][:c1] + self.lines[r2][c2:]
    after = self.lines[r2+1:]
    lines = before + [cur] + after
    return Buffer(lines, self.name)

In [493]:
b = Buffer(["two", "three", "four"])

test_eq(b.delete(2, 2, 0).lines, b.lines)

test_eq(b.delete(0, 1, 2).lines, ["t", "three", "four"])
test_eq(b.delete(0, 1, 3).lines, ["tthree", "four"])
test_eq(b.delete(0, 1, 10).lines, ["tour"])

test_eq(b.delete(2, 2, -1).lines, ["two", "three", "fur"])
test_eq(b.delete(2, 2, -5).lines, ["two", "thrur"])
test_eq(b.delete(2, 2, -10).lines, ["twur"])

## Cursor

> `Cursor`s move through `Buffer`s (which is tricky because line lengths are not uniform) and determine where `Buffer` operations occur.

In [516]:
class Cursor:
    def __init__(self, row=0, col=0, col_hint=None):
        col_hint = col if col_hint is None else col_hint
        store_attr()

    __repr__ = basic_repr("row,col,col_hint")
    
    def __eq__(self, o):
        if isinstance(o, self.__class__):
            return vars(self) == vars(o)
#             return (self.x, self.y, self.x_hint) == (o.x, o.y, o.x_hint)
        return False

In [518]:
c = Cursor()

test_eq(Cursor(), Cursor(0, 0, 0))
test_eq(Cursor(0, 1), Cursor(0, 1, 1))

Since `Buffer`s line lengths are not uniform, cursors often need to be "clamped" so that their position remains within a `Buffer`.

In [519]:
#export
def clamp(x, lower, upper):
    "Clamp x to the inclusive range [lower, upper]."
    if x < lower: return lower
    if x > upper: return upper
    return x

In [520]:
test_eq(clamp(0, 1, 5), 1)
test_eq(clamp(1, 1, 5), 1)
test_eq(clamp(2, 1, 5), 2)
test_eq(clamp(5, 1, 5), 5)
test_eq(clamp(6, 1, 5), 5)

In [527]:
#export
@patch
def column_move(self:Cursor, n, buffer):
    col = clamp(self.col+n, 0, len(buffer.lines[self.row]))
    return Cursor(self.row, col, col)

In [532]:
c = Cursor(1, 1)
b = Buffer(["foo", "bar", "baz"])

test_eq(c.column_move(1, b), Cursor(1, 2))
test_eq(c.column_move(3, b), Cursor(1, 3))
test_eq(c.column_move(-1, b), Cursor(1, 0))
test_eq(c.column_move(-3, b), Cursor(1, 0))

In [543]:
#export
@patch
def line_move(self:Cursor, n, buffer):
    row = clamp(self.row+n, 0, len(buffer.lines)-1)
    col = min(self.col_hint, len(buffer.lines[row]))
    return Cursor(row, col, self.col_hint)

In [544]:
c = Cursor(1, 1)
b = Buffer(["foo", "bar", "baz"])

test_eq(c.line_move(1, b), Cursor(2, 1))
test_eq(c.line_move(2, b), Cursor(2, 1))
test_eq(c.line_move(-1, b), Cursor(0, 1))
test_eq(c.line_move(-2, b), Cursor(0, 1))

In [545]:
b = Buffer(["foobar", "baz", "foobar"])
c1 = Cursor(0, 5)
c2 = c1.line_move(1, b2)
c3 = c2.line_move(1, b2)

test_eq(c2, Cursor(1, 3, 5))
test_eq(c3, Cursor(2, 5, 5))

In [546]:
#export
@patch
def up(self:Cursor, buffer):
    return self.line_move(-1, buffer)

@patch
def down(self:Cursor, buffer):
    return self.line_move(1, buffer)

@patch
def left(self:Cursor, buffer):
    if self.col == 0 and self.row > 0:
        return self.up(buffer).move_end_of_line(buffer)
    return self.column_move(-1, buffer)

@patch
def right(self:Cursor, buffer):
    if self.col == len(buffer.lines[self.row]) and self.row < len(buffer.lines):
        return self.down(buffer).move_beginning_of_line()
    return self.column_move(1, buffer)

@patch
def move_beginning_of_line(self:Cursor):
    return Cursor(0, self.row, 0)

@patch
def move_end_of_line(self:Cursor, buffer):
    return Cursor(len(buffer.lines[self.row]), self.row, self.MAX_X)

## Window

In [549]:
class Window:
    def __init__(self, nrows, ncols, row_offset, col_offset):
        store_attr()

    @property
    def row_end(self) -> int:
        return self.row + self.nrows

    def visible_lines(self, buffer):
        return buffer[self.row : self.row_end]

    def cursor_position(self, cursor):
        return cursor.col - self.col_offset, cursor.row - self.row_offset

    def scroll_up(self, cursor, margin=1):
        if self.row_offset > 0 and cursor.row < self.row_offset + margin:
            self.row_offset -= 1

    def scroll_down(self, cursor, buffer, margin=1):
        if self.row_end < len(buffer) and cursor.row >= self.row_end - margin:
            self.row_offset += 1

    def status_line(self, buffer, cursor):
        file_status = " " + buffer.filename
        if buffer.is_modified:
            file_status += " [+]"
        cursor_status = f"L: {cursor.row + 1}/{len(buffer)} C: {cursor.col + 1}" + " "
        pad_length = self.ncols - len(file_status) - len(cursor_status) - 1
        return file_status + " " * pad_length + cursor_status

## Blocks

> Outliner shit

- Each block contains a single buffer. Maybe it _is_ a buffer?
- Blocks have indentation levels. They exist in a tree.
- Can be edited.
- Can be moved.
- Can be nested / unnested.
- Can be expanded / collapsed.

In [589]:
class Page:
    def __init__(self, name, children):
        store_attr()

    def __str__(self):
        return "\n".join((self.name, len(self.name)*"-", *(str(c) for c in self.children)))

In [590]:
class Block:
    def __init__(self, string, children):
        store_attr()

    def print(self, level=0, indent=2, bullet="-"):
        indent = level * indent * " "
        string = indent + bullet + " " + self.string + "\n"
        for c in self.children:
            string += c.print(level + 1)
        return string

    def __str__(self):
        return self.print().strip()

In [591]:
b1 = Block("baz", [])
b2 = Block("bar", [b1])
b3 = Block("foo", [b2])

test_eq(str(b1), "- baz")
test_eq(str(b2), "- bar\n  - baz")
test_eq(str(b3), "- foo\n  - bar\n    - baz")

Can only be indented 1 level deeper than its previous block. How to state that in terms of children data structure?

Seems super tricky because of the orderedness...

In [592]:
@patch
def indent(self:Block, n):
    pass

In [613]:
print("Indent bar +1\n")

b1 = Block("baz", [])
b2 = Block("bar", [])
b3 = Block("foo", [b2, b1])

print(Page("Before", [b3]))

# b1.indent(1)

b1 = Block("baz", [])
b2 = Block("bar", [b1])
b3 = Block("foo", [b2])

print()
print(Page("After", [b3]))

# b1 is popped from its parent to its latest sibling

Indent bar +1

Before
------
- foo
  - bar
  - baz

After
-----
- foo
  - bar
    - baz


In [609]:
print("Indent bar -1\n")

b1 = Block("baz", [])
b2 = Block("bar", [b1])
b3 = Block("foo", [b2])

print(Page("Before", [b3]))

# b2.indent(-1)

b1 = Block("baz", [])
b2 = Block("bar", [b1])
b3 = Block("foo", [])

print()
print(Page("After", [b3, b2]))

# b2 gets popped from its parent to its parent's parent

Indent bar -1

Before
------
- foo
  - bar
    - baz

After
-----
- foo
- bar
  - baz


In [574]:
@patch
def move(self:Block, n):
    pass

Can only move if it's valid to be at the same indentation at the target.

In [618]:
print("Move bar -1\n")

b1 = Block("baz", [])
b2 = Block("bar", [b1])
b3 = Block("foo", [])

print(Page("Before", [b3, b2]))

# b2.move(-1)

b1 = Block("baz", [])
b2 = Block("bar", [b1])
b3 = Block("foo", [])

print()
print(Page("After", [b2, b3]))

# b2 is moved to before its previous sibling.
# If there are no further siblings, try to move to the last child of the parent's previous sibling.

Move bar -1

Before
------
- foo
- bar
  - baz

After
-----
- bar
  - baz
- foo


Are expand/collapse just binary flags that are only used at the view layer? I guess I can simulate with `__str__`.

In [574]:
@patch
def expand(self:Block):
    pass

In [575]:
@patch
def collapse(self:Block):
    pass

In [577]:
# TODO: New block? Where does that go?

# TODO: Delete block? Where does that go?

## Pages

In [551]:
class Page:
    def __init__(self, name, children):
        store_attr()

## Roamlike Parser

## IPython Kernel Wrapper

## Keys

## Editor

In [ ]:
def main(argv: Optional[Sequence[str]] = None) -> int:
    parser = argparse.ArgumentParser()
    parser.add_argument("filename")
    args = parser.parse_args(argv)
    return curses.wrapper(c_main, args.filename)

In [ ]:
def c_main(stdscr: "curses._CursesWindow", filename: str) -> int:
    Editor(filename).run(stdscr)
    return 0

In [ ]:
class Editor:
    def __init__(self, filename: str):
        with open(filename) as f:
            lines = f.read().split("\n")
        self.buffer = Buffer(lines, filename)
        self.cursor = Cursor()
        self.window = Window(curses.COLS, curses.LINES - 1)
        self.undo_stack: List[Tuple[Buffer, Cursor]] = []
        self.redo_stack: List[Tuple[Buffer, Cursor]] = []
        self.command_line = ""

    # Main

    def run(self, stdscr: "curses._CursesWindow") -> None:
        while True:
            self.render(stdscr)
            self.command_line = ""
            self.handle_key(stdscr)

    def render(self, stdscr: "curses._CursesWindow") -> None:
        stdscr.erase()
        # Window
        for y, line in enumerate(self.window.visible_lines(self.buffer)):
            stdscr.addstr(y, 0, line)
        # Status line
        stdscr.addstr(
            self.window.height - 1,
            0,
            self.window.status_line(self.buffer, self.cursor),
            curses.A_REVERSE,
        )
        # Command line
        stdscr.addstr(self.window.height, 0, self.command_line)
        # Cursor
        stdscr.move(*self.window.cursor_position(self.cursor))

    def handle_key(self, stdscr: "curses._CursesWindow") -> None:
        c = self.getkey(stdscr)

        with open("log.txt", "a") as f:
            f.write(c + "\n")

        if c == "C-q":
            self.exit()
        elif c == "C-p":
            self.previous_line()
        elif c == "C-n":
            self.next_line()
        elif c == "C-b":
            self.backward_char()
        elif c == "C-f":
            self.forward_char()
        elif c == "C-a":
            self.move_beginning_of_line()
        elif c == "C-e":
            self.move_end_of_line()
        elif c == "C-j":  # <enter>
            self.newline()
        elif c == "<backspace>":
            self.delete_char()
        elif c == "C-d":  # del
            self.delete_forward_char()
        elif c == "C-s":
            self.save_buffer()
        elif c == "C-_":  # C-/
            self.undo()
        elif c == "M-/":
            self.redo()
        else:
            self.add_char(c)

    def exit(self) -> None:
        # TODO: If any buffer has been modified, print an error message instead.
        # TODO: Add a prompt that waits for the user to press enter to continue.
        if self.buffer.is_modified:
            self.send_message(
                f'No write since last change for buffer "{self.buffer.filename}"',
            )
            return
        sys.exit(0)

    # Keyboard

    def getkey(self, stdscr: "curses._CursesWindow") -> str:
        # TODO: Make a simple Key class that knows that some keys
        #       have multiple possible chars. E.g., C-j = RET = ...
        c = stdscr.getch()

        # Meta
        if c == curses.ascii.ESC:
            stdscr.nodelay(True)
            c2 = stdscr.getch()
            stdscr.nodelay(False)
            if c2 == curses.ERR:  # no additional key pressed
                return "<escape>"
            # Ctrl + Meta
            c2_unctrl = curses.unctrl(c2).decode("ascii")
            if curses.ascii.isctrl(c2):
                c2_key = c2_unctrl[1:].lower()
                return f"C-M-{c2_key}"
            return f"M-{c2_unctrl}"

        # Ctrl
        if curses.ascii.isctrl(c):
            c_key = curses.unctrl(c).decode("ascii")[1:].lower()
            return f"C-{c_key}"

        if c == curses.ascii.DEL:
            return "<backspace>"

        # Plain char
        if curses.ascii.isprint(c):
            return curses.unctrl(c).decode("ascii")

        raise NotImplementedError(f"Unknown character key code: {c}")

    # Cursor movement

    def previous_line(self) -> None:
        self.cursor = self.cursor.up(self.buffer)
        self.window.scroll_up(self.cursor)

    def next_line(self) -> None:
        self.cursor = self.cursor.down(self.buffer)
        self.window.scroll_down(self.cursor, self.buffer)

    def backward_char(self) -> None:
        self.cursor = self.cursor.left(self.buffer)
        self.window.scroll_up(self.cursor)

    def forward_char(self) -> None:
        self.cursor = self.cursor.right(self.buffer)
        self.window.scroll_down(self.cursor, self.buffer)

    # Buffer editing

    def delete_char(self) -> None:
        if not (self.cursor.y == 0 and self.cursor.x == 0):
            self._checkpoint()
            # Move the cursor with the unmodified buffer, else it ends up at the
            # end of the new, combined line.
            cursor = self.cursor.left(self.buffer)
            self.buffer = self.buffer.delete_char(self.cursor)
            self.cursor = cursor
            self.window.scroll_up(self.cursor)

    def delete_forward_char(self) -> None:
        if not (
            self.cursor.y == len(self.buffer) - 1
            and self.cursor.x == len(self.buffer[self.cursor.y])
        ):
            self._checkpoint()
            self.buffer = self.buffer.delete_forward_char(self.cursor)

    def add_char(self, c: str) -> None:
        self._checkpoint()
        self.buffer = self.buffer.add_char(self.cursor, c)
        self.cursor = self.cursor.right(self.buffer)

    def newline(self) -> None:
        self._checkpoint()
        self.buffer = self.buffer.newline(self.cursor)
        self.cursor = self.cursor.right(self.buffer)
        self.window.scroll_down(self.cursor, self.buffer)

    def move_beginning_of_line(self) -> None:
        self.cursor = self.cursor.move_beginning_of_line()

    def move_end_of_line(self) -> None:
        self.cursor = self.cursor.move_end_of_line(self.buffer)

    # IO

    def save_buffer(self) -> None:
        # TODO: This isn't safe! We should check if the file changed externally.
        self.buffer.save()
        self.send_message(f'"{self.buffer.filename}" {len(self.buffer)}L written')

    # Undo/redo

    def _checkpoint(self) -> None:
        self.redo_stack = []
        self.undo_stack.append((self.buffer, self.cursor))

    def undo(self) -> None:
        if self.undo_stack:
            self.redo_stack.append((self.buffer, self.cursor))
            self.buffer, self.cursor = self.undo_stack.pop()
        else:
            self.send_message("Already at oldest change")

    def redo(self) -> None:
        if self.redo_stack:
            self.undo_stack.append((self.buffer, self.cursor))
            self.buffer, self.cursor = self.redo_stack.pop()
        else:
            self.send_message("Already at newest change")

    # Messages

    def send_message(self, message: str) -> None:
        self.command_line = message